In [8]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

In [9]:
mnist_dataset, mnist_info = tfds.load(name='mnist',with_info=True,as_supervised=True)

mnist_train ,mnist_test = mnist_dataset['train'], mnist_dataset['test']

num_validation_samples = 0.1* mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

def scale(image,label):
    image = tf.cast(image, tf.float32)
    image /= 155.
    return image, label

scaled_train_and_validation_data = mnist_train.map(scale)

test_data = mnist_test.map(scale)

BUFFER_SIZE= 10000

shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

BATCH_SIZE = 100 
train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)

test_data = test_data.batch(num_test_samples)

validation_inputs, validation_targets = next(iter(validation_data))

In [10]:
input_size = 784
output_size = 10
hidden_layer_size = 100

model = tf.keras.Sequential([
                            tf.keras.layers.Flatten(input_shape=(28,28,1)),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(output_size, activation='softmax')
])

In [11]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [12]:
NUM_EPOCHS = 5

model.fit(train_data,epochs=NUM_EPOCHS, validation_data=(validation_inputs, validation_targets), verbose =2)

Epoch 1/5
540/540 - 3s - loss: 0.2999 - accuracy: 0.9124 - val_loss: 0.1619 - val_accuracy: 0.9480 - 3s/epoch - 6ms/step
Epoch 2/5
540/540 - 2s - loss: 0.1250 - accuracy: 0.9629 - val_loss: 0.1116 - val_accuracy: 0.9685 - 2s/epoch - 4ms/step
Epoch 3/5
540/540 - 3s - loss: 0.0860 - accuracy: 0.9740 - val_loss: 0.0790 - val_accuracy: 0.9780 - 3s/epoch - 5ms/step
Epoch 4/5
540/540 - 2s - loss: 0.0673 - accuracy: 0.9791 - val_loss: 0.0720 - val_accuracy: 0.9778 - 2s/epoch - 4ms/step
Epoch 5/5
540/540 - 3s - loss: 0.0545 - accuracy: 0.9834 - val_loss: 0.0550 - val_accuracy: 0.9823 - 3s/epoch - 5ms/step


In [14]:
test_loss, test_accuracy = model.evaluate(test_data)


1/1 [==============================] - 0s 97ms/step - loss: 0.0797 - accuracy: 0.9738


In [15]:
print("Test loss:  {0:.2f}. Test Accuracy: {1:.2f}%".format(test_loss,test_accuracy*100))

Test loss:  0.08. Test Accuracy: 97.38%
